## Initialize: read data and generate essential data structures

In [39]:
import numpy as np
import json
from collections import defaultdict
from sklearn import metrics
import pandas as pd

labels42 = json.load(open('/home/yuhuiz/Transformer/data/csu/snomed_labels.json'))
labels = json.load(open('/home/yuhuiz/Transformer/data/csu/snomed_fine_grained_labels.json'))
label2id = dict((j, i) for i, j in enumerate(labels))
names = json.load(open('/home/yuhuiz/Transformer/data/csu/snomed_fine_grained_labels_to_name.json'))

In [40]:
f = open('/home/yuhuiz/csu/tree.txt')
father = {}
label2depth = dict()
visited = set()
for line in f:
    line = line.strip()
    a, b, c, d = line.split('\t')
    if b in visited:
        continue
    visited.add(b)
    father[b] = a
    e = b
    s = [e]
    while e in father:
        e = father[e]
        s.append(e)
    label2depth[b] = len(s)
label2depth['64572001'] = 1
print np.mean(label2depth.values())

6.0125677863733324


In [41]:
x = defaultdict(list)
y = []
y_reverse = []
for i, label in enumerate(labels):
    if label in labels42: y.append(i)
    else: y_reverse.append(i)
    x[label2depth.get(label, -1)].append(i)

In [42]:
# Clinical finding not in 4577 labels!
for k in labels42:
    if k not in labels:
        print k

404684003


## VetTag : performance on 41 diseases (original 42 diseases except Clinical Finding)

In [43]:
def process_preds(preds):
    parents = json.load(open('/home/yuhuiz/Transformer/data/parents.json')) # {child_did: parent_did, ..}
    id2label = json.load(open('/home/yuhuiz/Transformer/data/labels.json')) # [did1, ..]
    label2id = dict([(j, i) for i, j in enumerate(id2label)])
    for i in range(preds.shape[0]): 
        last_pred_i = preds[i].copy()
        while True:
            for j in range(preds.shape[1]):
                did = id2label[j]
                flag = True
                now = did
                while now in parents:
                    now = parents[now]
                    if now not in label2id: break
                    if preds[i, label2id[now]] == 0:
                        flag = False
                        break
                if not flag: 
                    preds[i, j] = 0.
            if (preds[i] == last_pred_i).all(): break
            last_pred_i = preds[i].copy()
    return preds

In [44]:
goldens_csu = np.load('/home/yuhuiz/Transformer/data/csu/finegrained_labels.npy')
goldens_pp = np.load('/home/yuhuiz/Transformer/data/pp/finegrained_labels.npy')
preds_csu = (np.load('/home/yuhuiz/Transformer/exp/analysis/pp/taph.csu2.npy') > 0.5).astype(float)
preds_pp = (np.load('/home/yuhuiz/Transformer/exp/analysis/pp/taph.pp2.npy') > 0.5).astype(float)
preds_csu = process_preds(preds_csu)
preds_pp = process_preds(preds_pp)
em_csu = metrics.accuracy_score(goldens_csu, preds_csu)
em_pp = metrics.accuracy_score(goldens_pp, preds_pp)

print em_csu, em_pp

0.2615493958777541 0.09215017064846416


In [45]:
print'\tEM\tP\tR\tF1'
goldens_ = goldens_csu[:, y]
preds_ = preds_csu[:, y]
em = metrics.accuracy_score(goldens_, preds_)
p, r, f1, s = metrics.precision_recall_fscore_support(goldens_, preds_, average='weighted')
print 'TAPF\t%.3f\t%.3f\t%.3f\t%.3f' % (em, p, r, f1)

	EM	P	R	F1
TAPF	0.493	0.798	0.691	0.736


In [46]:
print'\tEM\tP\tR\tF1'
goldens_ = goldens_pp[:, y]
preds_ = preds_pp[:, y]
em = metrics.accuracy_score(goldens_, preds_)
p, r, f1, s = metrics.precision_recall_fscore_support(goldens_, preds_, average='weighted')
print 'TAPF\t%.3f\t%.3f\t%.3f\t%.3f' % (em, p, r, f1)

	EM	P	R	F1
TAPF	0.232	0.612	0.510	0.538


## VetTag: how hierachical loss improve the result?

In [49]:
preds = (np.load('/home/yuhuiz/Transformer/exp/analysis/pp/tap.csu.npy') > 0.5).astype(float)
p1, r1, f11, s1 = metrics.precision_recall_fscore_support(goldens_csu, preds, average='weighted')
p2, r2, f12, s2 = metrics.precision_recall_fscore_support(goldens_csu, preds_csu, average='weighted')
print 'P\tR\tF1\tP\tR\tF1'
print '%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f' % (p1, r1, f11, p2, r2, f12)

print 'Depth\tNumber\tF1\tP\tR\tEM\tF1\tP\tR\tEM'
for i in range(1, 13):
    goldens_ = goldens_csu[:, x[i]]
    if goldens_.shape[1] == 0: continue
    
    preds_ = preds[:, x[i]]
    em1 = metrics.accuracy_score(goldens_, preds_)
    p1, r1, f11, s1 = metrics.precision_recall_fscore_support(goldens_, preds_, average='weighted')
    
    preds_ = preds_csu[:, x[i]]
    em2 = metrics.accuracy_score(goldens_, preds_)
    p2, r2, f12, s2 = metrics.precision_recall_fscore_support(goldens_, preds_, average='weighted')
    
    print '%d\t%d\t%.1f\t%.1f\t%.1f\t%.1f\t%.1f\t%.1f\t%.1f\t%.1f' % (i - 1, goldens_.shape[1], 100*f11, 100*p1, 100*r1, 100*em1, 100*f12, 100*p2, 100*r2, 100*em2)  

P	R	F1	P	R	F1
0.744	0.598	0.648	0.721	0.631	0.662
Depth	Number	F1	P	R	EM	F1	P	R	EM
1	56	76.2	81.8	74.1	51.3	76.6	83.8	72.9	52.7
2	299	73.3	79.1	70.0	35.9	73.8	78.3	71.3	38.9
3	632	66.9	75.9	61.1	31.0	68.1	72.9	65.1	33.3
4	1086	62.6	73.3	56.7	33.7	63.9	69.7	60.9	33.8
5	1298	55.6	68.4	49.8	45.8	57.7	65.2	54.2	44.1
6	804	45.2	62.7	39.7	68.2	49.4	59.2	45.7	65.5
7	283	37.9	54.7	31.1	90.2	45.3	56.1	43.3	89.7
8	66	19.7	41.9	14.4	97.4	31.7	44.1	31.5	97.4
9	6	0.0	0.0	0.0	100.0	0.0	0.0	0.0	100.0
10	2	0.0	0.0	0.0	100.0	0.0	0.0	0.0	100.0


In [50]:
preds = (np.load('/home/yuhuiz/Transformer/exp/analysis/pp/tap.pp.npy') > 0.5).astype(float)
p1, r1, f11, s1 = metrics.precision_recall_fscore_support(goldens_pp, preds, average='weighted')
p2, r2, f12, s2 = metrics.precision_recall_fscore_support(goldens_pp, preds_pp, average='weighted')
print 'P\tR\tF1\tP\tR\tF1'
print '%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f' % (p1, r1, f11, p2, r2, f12)

print 'Depth\tNumber\tF1\tP\tR\tEM\tF1\tP\tR\tEM'
for i in range(1, 13):
    goldens_ = goldens_pp[:, x[i]]
    if goldens_.shape[1] == 0: continue
    
    preds_ = preds[:, x[i]]
    em1 = metrics.accuracy_score(goldens_, preds_)
    p1, r1, f11, s1 = metrics.precision_recall_fscore_support(goldens_, preds_, average='weighted')
    
    preds_ = preds_pp[:, x[i]]
    em2 = metrics.accuracy_score(goldens_, preds_)
    p2, r2, f12, s2 = metrics.precision_recall_fscore_support(goldens_, preds_, average='weighted')
    
    print '%d\t%d\t%.1f\t%.1f\t%.1f\t%.1f\t%.1f\t%.1f\t%.1f\t%.1f' % (i - 1, goldens_.shape[1], 100*f11, 100*p1, 100*r1, 100*em1, 100*f12, 100*p2, 100*r2, 100*em2)  

P	R	F1	P	R	F1
0.531	0.427	0.450	0.549	0.477	0.486
Depth	Number	F1	P	R	EM	F1	P	R	EM
1	56	57.8	61.8	57.2	26.8	57.7	67.6	54.9	25.3
2	299	52.4	56.3	52.0	13.8	55.5	58.7	56.5	15.0
3	632	46.0	54.5	42.4	14.2	50.2	55.6	49.8	11.9
4	1086	43.4	54.7	39.3	16.9	46.9	54.1	45.2	14.5
5	1298	28.3	38.7	26.9	25.8	33.4	42.2	31.3	24.6
6	804	16.1	28.6	17.7	58.9	22.9	28.7	21.6	54.3
7	283	10.9	10.3	13.0	86.5	14.9	24.3	14.5	86.3
8	66	18.2	50.0	11.1	95.1	0.0	0.0	0.0	97.8
9	6	0.0	0.0	0.0	98.6	0.0	0.0	0.0	100.0
10	2	0.0	0.0	0.0	98.8	0.0	0.0	0.0	100.0


## How many diseases in different depths and what they are?

In [51]:
csu_label_train = []
with open('/home/yuhuiz/Transformer/data/csu/csu_bpe_finegrained_train.tsv') as fin:
    for line in fin:
        splitline = line.strip().split('\t')
        csu_label_train.append([] if len(splitline) == 1 else map(int, splitline[1].split()))
        
csu_label_valid = []
with open('/home/yuhuiz/Transformer/data/csu/csu_bpe_finegrained_valid.tsv') as fin:
    for line in fin:
        splitline = line.strip().split('\t')
        csu_label_valid.append([] if len(splitline) == 1 else map(int, splitline[1].split()))
        
csu_label_test = []
with open('/home/yuhuiz/Transformer/data/csu/csu_bpe_finegrained_test.tsv') as fin:
    for line in fin:
        splitline = line.strip().split('\t')
        csu_label_test.append([] if len(splitline) == 1 else map(int, splitline[1].split()))
        
pp_label_test = []
with open('/home/yuhuiz/Transformer/data/pp/pp_bpe_finegrained_test.tsv') as fin:
    for line in fin:
        splitline = line.strip().split('\t')
        pp_label_test.append([] if len(splitline) == 1 else map(int, splitline[1].split()))

print len(csu_label_train), len(csu_label_valid), len(csu_label_test), len(pp_label_test)

101301 5628 5628 586


In [52]:
disease2cnt_csu = defaultdict(int)

for s in csu_label_train + csu_label_valid + csu_label_test:
    for d in s:
        disease2cnt_csu[d] += 1
        
disease2cnt_pp = defaultdict(int)

for s in pp_label_test:
    for d in s:
        disease2cnt_pp[d] += 1

In [53]:
for i in range(1, 13):
    print
    ds = {}
    for d in x[i]:
        ds[d] = disease2cnt_csu[d]
    for item in sorted(ds.items(), key=lambda x:x[1], reverse=True)[:5]:
        did = int(item[0])
        
        goldens_csu_ = goldens_csu[:, [did, 0]]
        preds_csu_ = preds_csu[:, [did, 0]]
        em_csu = metrics.accuracy_score(goldens_csu_[:, 0], preds_csu_[:, 0])
        p_csu, r_csu, f1_csu, s_csu = metrics.precision_recall_fscore_support(goldens_csu_, preds_csu_, average=None)
        goldens_pp_ = goldens_pp[:, [did, 0]]
        preds_pp_ = preds_pp[:, [did, 0]]
        em_pp = metrics.accuracy_score(goldens_pp_[:, 0], preds_pp_[:, 0])
        p_pp, r_pp, f1_pp, s_pp = metrics.precision_recall_fscore_support(goldens_pp_, preds_pp_, average=None)

        print '%d\t%.1f\t%.1f\t%.1f\t%.1f\t%d\t%.1f\t%.1f\t%.1f\t%.1f\t%s' % (disease2cnt_csu[did], 100*f1_csu[0], 100*p_csu[0], 100*r_csu[0], 100*em_csu, disease2cnt_pp[did], 100*f1_pp[0], 100*p_pp[0], 100*r_pp[0], 100*em_pp, names[did])  



84832	91.1	90.4	91.8	86.6	461	82.9	84.3	81.6	73.5	Disease by body site (disorder)
36108	90.5	93.0	88.1	94.0	59	48.5	37.3	69.5	85.2	Neoplasm and/or hamartoma
25271	72.3	77.4	67.8	88.6	193	64.0	72.8	57.0	78.8	Inflammatory disorder (disorder)
11304	59.5	69.6	52.1	92.7	88	42.2	67.5	30.7	87.4	Infectious disease (disorder)
9722	46.5	73.3	34.0	93.5	38	27.5	53.8	18.4	93.7	Degenerative disorder (disorder)

79365	89.5	88.3	90.8	85.2	459	82.8	84.5	81.0	73.5	Disorder of body system (disorder)
36237	75.4	78.1	73.0	85.1	205	65.1	56.9	76.1	71.5	Disorder of soft tissue (disorder)
36095	90.5	93.0	88.1	94.0	59	48.5	37.3	69.5	85.2	Neoplasm
35398	77.9	77.0	78.8	86.1	147	55.9	53.0	59.2	76.6	Disease of trunk (disorder)
31524	87.2	88.6	85.8	93.0	28	25.5	15.7	67.9	81.1	Neoplasm by body site (disorder)

28058	91.1	93.0	89.2	95.6	19	51.5	36.2	89.5	94.5	Malignant tumor (disorder)
23911	71.8	70.8	72.9	88.1	190	66.1	65.5	66.8	77.8	Inflammation of specific body systems (disorder)
22589	70.5	75.6	66.0	89.3	195	58.

In [54]:
depth2cnt_csu = defaultdict(int)
for i in range(1, 13):
    for s in csu_label_train + csu_label_valid + csu_label_test:
        for d in s:
            if d in x[i]:
                depth2cnt_csu[i] += 1
                break
        
depth2cnt_pp = defaultdict(int)
for i in range(1, 13):
    for s in pp_label_test:
        for d in s:
            if d in x[i]:
                depth2cnt_pp[i] += 1
                break
                
print depth2cnt_csu

print depth2cnt_pp

defaultdict(<type 'int'>, {2: 91109, 3: 90880, 4: 89856, 5: 85783, 6: 70242, 7: 46250, 8: 12994, 9: 2918, 10: 66, 11: 9})
defaultdict(<type 'int'>, {2: 497, 3: 495, 4: 489, 5: 462, 6: 389, 7: 216, 8: 68, 9: 9})


## Performance by species

In [55]:
import json
examples = csu_species_freq = json.load(open('/home/yuhuiz/csu/species/csu_species_freq.json'))
np.random.seed(1234)
train_size = 0.9
split_proportions = {
    "train": train_size,
    "valid": (1 - train_size) / 2,
    "test": (1 - train_size) / 2
}
serial_numbers = range(len(examples))
np.random.shuffle(serial_numbers)

train_numbers = serial_numbers[:int(np.rint(len(examples) * split_proportions['train']))]
valid_numbers = serial_numbers[
    int(np.rint(len(examples) * split_proportions['train'])): \
    int(np.rint(len(examples) * (split_proportions['train'] + split_proportions['valid'])))]
test_numbers = serial_numbers[int(np.rint(len(examples) * (split_proportions['train'] + split_proportions['valid']))):]
print test_numbers[:10]

[84276, 40905, 94532, 102121, 75567, 20282, 19212, 97220, 90601, 15321]


In [56]:
test_csu_species = [csu_species_freq[i] for i in test_numbers]

In [ ]:
species2id = defaultdict(list)
for i, j in enumerate(test_csu_species):
    species2id[j].append(i)

In [ ]:
print 'CSU'
print 'SPECIES\t#\tEM\tP\tR\tF1'  
for i in species2id:
    goldens_csu_ = goldens_csu[species2id[i]]
    preds_csu_ = preds_csu[species2id[i]]
    em = metrics.accuracy_score(goldens_csu_, preds_csu_)
    p, r, f1, s = metrics.precision_recall_fscore_support(goldens_csu_, preds_csu_, average='weighted')
    print '%s\t%d\t%.3f\t%.3f\t%.3f\t%.3f' % (i, len(species2id[i]), em, p, r, f1)  

CSU
SPECIES	#	EM	P	R	F1
CAPRINE	21	0.381	0.368	0.450	0.395


In [ ]:
pp_species2id = defaultdict(list)
for i, j in enumerate(json.load(open('/home/yuhuiz/csu/species/pp_species_freq.json'))):
    pp_species2id[j].append(i)

In [ ]:
print 'PP'
print 'SPECIES\t#\tEM\tP\tR\tF1'  
for i in pp_species2id:
    goldens_pp_ = goldens_pp[pp_species2id[i]]
    preds_pp_ = preds_pp[pp_species2id[i]]
    em = metrics.accuracy_score(goldens_pp_, preds_pp_)
    p, r, f1, s = metrics.precision_recall_fscore_support(goldens_pp_, preds_pp_, average='weighted')
    print '%s\t%d\t%.3f\t%.3f\t%.3f\t%.3f' % (i, len(pp_species2id[i]), em, p, r, f1)  